<a href="https://colab.research.google.com/github/mmmshakeel/NB-ensemle-ddos-prediction/blob/main/Naive_Bayes_Ensemble_Prediction_with_2_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import mode
from google.colab import drive

In [ ]:
# Mount Google drive to load datasets
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Load the dataset (adjust this part according to your dataset format and file location)
df = pd.read_csv('/content/gdrive/MyDrive/[03] Study/Colab Notebooks/Datasets/CICIoT_2023_2_labels/part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv')
print(df.head())

   Unnamed: 0  flow_duration  Header_Length  Protocol Type  Duration  \
0           0       0.000000          54.00           6.00     64.00   
1           1       0.000000          57.04           6.33     64.00   
2           2       0.000000           0.00           1.00     64.00   
3           3       0.328175       76175.00          17.00     64.00   
4           4       0.117320         101.73           6.11     65.91   

          Rate        Srate  Drate  fin_flag_number  syn_flag_number  ...  \
0     0.329807     0.329807    0.0              1.0              0.0  ...   
1     4.290556     4.290556    0.0              0.0              0.0  ...   
2    33.396799    33.396799    0.0              0.0              0.0  ...   
3  4642.133010  4642.133010    0.0              0.0              0.0  ...   
4     6.202211     6.202211    0.0              0.0              1.0  ...   

         Std  Tot size           IAT  Number   Magnitue     Radius  \
0   0.000000     54.00  8.334383e+

In [ ]:
# List of selected features
selected_features = ['IAT', 'Tot sum', 'Tot size', 'AVG', 'flow_duration', 'Magnitue',
                     'Header_Length', 'Max', 'Min', 'Protocol Type', 'Rate', 'Srate', 'Radius',
                     'Covariance', 'rst_count', 'urg_count', 'Duration', 'Weight', 'Std', 'ICMP',
                     'Variance', 'ack_flag_number', 'Number', 'UDP', 'syn_count', 'fin_count']

target_variable = 'label'

# Number of models in the ensemble
n_models = 10
models = [GaussianNB() for _ in range(n_models)]

os.chdir('/content/gdrive/MyDrive/[03] Study/Colab Notebooks/Datasets/CICIoT_2023_2_labels/');

# For each dataset...
for filename in os.listdir():
    if filename.endswith('.csv'):
        # Read each csv file and append to list
        df = pd.read_csv(filename)

        # Extract the features and target variable
        X = df[selected_features]
        y = df[target_variable]

        # Split the dataset into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # If it's the first dataset, initialize models
        if not models:
            models = [GaussianNB() for _ in range(n_models)]

        # Train each model on the dataset
        for model in models:
            boot_indices = np.random.randint(0, len(X_train), len(X_train))
            X_boot = X_train.iloc[boot_indices]
            y_boot = y_train.iloc[boot_indices]
            model.fit(X_boot, y_boot)


In [ ]:
from statistics import mode

os.chdir('/content/gdrive/MyDrive/[03] Study/Colab Notebooks/Datasets/CICIoT_2023_2_labels/');
filename = 'part-00028-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv'

# Make predictions with each model
test_df = pd.read_csv(filename)

X_test = test_df[selected_features]
y_test = test_df[target_variable]

predictions = [model.predict(X_test) for model in models]

# Compute the mode of the predictions
predictions_transposed = np.array(predictions).T
ensemble_predictions = [mode(pred) for pred in predictions_transposed]

ensemble_predictions_np = np.array(ensemble_predictions)

# Evaluating the ensemble model
accuracy = accuracy_score(y_test, ensemble_predictions_np)
print(f"Ensemble Accuracy: {accuracy}")
print(classification_report(y_test, ensemble_predictions_np))


Ensemble Accuracy: 0.7668981988257538
              precision    recall  f1-score   support

        DDoS       0.77      0.97      0.86    183070
    Non-DDoS       0.72      0.23      0.35     68155

    accuracy                           0.77    251225
   macro avg       0.75      0.60      0.60    251225
weighted avg       0.76      0.77      0.72    251225



In [ ]:
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder

predictions2 = np.array(predictions)

# Use label encoding to convert string class labels to numerical labels
label_encoder = LabelEncoder()
predictions_encoded = np.apply_along_axis(label_encoder.fit_transform, 1, predictions2)

# Use mode to find the most common prediction for each instance
majority_vote, _ = mode(predictions_encoded, axis=0)

# Flatten the array to get a 1D list of predictions
majority_vote = majority_vote.flatten()

# Decode the numerical labels back to original string labels
majority_vote_labels = label_encoder.inverse_transform(majority_vote)

# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(y_test, majority_vote_labels)
print(f'Ensemble accuracy: {ensemble_accuracy:.2f}')
print(f'Ensemble accuracy: {ensemble_accuracy}')
print(classification_report(y_test, majority_vote_labels, zero_division=0))

Ensemble accuracy: 0.77
Ensemble accuracy: 0.7666855917234419
              precision    recall  f1-score   support

        DDoS       0.77      0.97      0.86    161784
    Non-DDoS       0.72      0.23      0.35     60144

    accuracy                           0.77    221928
   macro avg       0.75      0.60      0.60    221928
weighted avg       0.76      0.77      0.72    221928



In [ ]:
import joblib

os.chdir('/content/gdrive/MyDrive/[03] Study/Colab Notebooks/Trained Models')

# Save each model to disk
for idx, model in enumerate(models):
    filename = f'naive_bayes_model_{idx}.joblib'
    joblib.dump(model, filename)
